In [1]:
import gym
import ray

In [2]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [3]:
# Initialize Ray
ray.shutdown()
ray.init(ignore_reinit_error=True,num_cpus=8, num_gpus=1)

2020-09-27 21:14:52,746	INFO resource_spec.py:231 -- Starting Ray with 12.79 GiB memory available for workers and up to 6.4 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-27 21:14:53,291	INFO services.py:1193 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '192.168.7.73',
 'raylet_ip_address': '192.168.7.73',
 'redis_address': '192.168.7.73:36738',
 'object_store_address': '/tmp/ray/session_2020-09-27_21-14-52_745012_57185/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-09-27_21-14-52_745012_57185/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-09-27_21-14-52_745012_57185'}

In [4]:
# https://docs.ray.io/en/latest/tune/user-guide.html#parallelism-gpus
ray.cluster_resources()

{'object_store_memory': 90.0,
 'GPUType:GTX': 1.0,
 'CPU': 8.0,
 'node:192.168.7.73': 1.0,
 'memory': 262.0,
 'GPU': 1.0}

In [5]:
print("Dashboard URL: http://{}".format(ray.get_webui_url()))

Dashboard URL: http://localhost:8266


### Register MineRL Gym Environment to RLlib

In [3]:
from minerl_rllib.envs import register

In [4]:
register()

## Example of registering a custom model

In [6]:
# Registering a custom model, simple version --> fully connected network
# We need this because the default configurations for the model network is
# not compatible with the shape of the output from the MineRL environment
# The output is a 64 x 64 pixels with RGBA of game play P.O.V. --> shape = (4, 64, 64)

# Otherwise we will get:
# ValueError: No default configuration for obs shape [4, 64, 64], you must specify 
# `conv_filters` manually as a model option. Default configurations are only available
# for inputs of shape [42, 42, K] and [84, 84, K]. You may alternatively want to use 
# a custom model or preprocessor.

from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC
import torch.nn as nn
# https://docs.ray.io/en/releases-0.8.5/rllib-examples.html
# The register custom env and model links to custom_env.py
# https://github.com/ray-project/ray/blob/master/rllib/examples/custom_env.py
class TorchCustomModel(TorchModelV2, nn.Module):
    """Example of a PyTorch custom model that just delegates to a fc-net."""

    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs,
                              model_config, name)
        nn.Module.__init__(self)

        self.torch_sub_model = TorchFC(obs_space, action_space, num_outputs,
                                       model_config, name)

    def forward(self, input_dict, state, seq_lens):
        input_dict["obs"] = input_dict["obs"].float()
        fc_out, _ = self.torch_sub_model(input_dict, state, seq_lens)
        return fc_out, []

    def value_function(self):
        return torch.reshape(self.torch_sub_model.value_function(), [-1])


In [7]:
ModelCatalog.register_custom_model("fc_pov", TorchCustomModel)

## Run training

In [10]:
from ray import tune
# https://docs.ray.io/en/latest/rllib-training.html#tuned-examples

import ray.rllib.agents.ppo as ppo
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.dqn import DQNTrainer

tune.run(DQNTrainer,
         config={"env": "MineRLNavigateDenseVectorObf-v0",
                 "num_gpus": 1,
                 "num_workers": 1,
                 "num_envs_per_worker": 1,
                 "eager": False,
                 "use_pytorch": True,
                 'monitor':True, 
#                  "model": {
#                     "custom_model": "fc_pov",
#                     }
                 }
         ,stop={"training_iteration": 3})  

Trial name,status,loc
DQN_MineRLNavigateDenseVectorObf-v0_2b461_00000,RUNNING,


(pid=57257) WARNING:tensorflow:From /home/blackbox/anaconda3/envs/minerl/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=57257) Instructions for updating:
(pid=57257) non-resource variables are not supported in the long term
(pid=57257) /home/blackbox/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=57257)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2020-09-27 21:15:00,442	ERROR trial_runner.py:523 -- Trial DQN_MineRLNavigateDenseVectorObf-v0_2b461_00000: Error processing event.
Traceback (most recent call last):
  File "/home/blackbox/anaconda3/envs/minerl/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 471, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/home/blackbox/ana

Trial name,status,loc
DQN_MineRLNavigateDenseVectorObf-v0_2b461_00000,ERROR,
Trial name,# failures,error file
DQN_MineRLNavigateDenseVectorObf-v0_2b461_00000,1,/home/blackbox/ray_results/DQN/DQN_MineRLNavigateDenseVectorObf-v0_0_2020-09-27_21-14-571_cgcmhc/error.txt


(pid=57257) 2020-09-27 21:15:00,424	WARNING deprecation.py:30 -- DeprecationWarning: `use_pytorch` has been deprecated. Use `framework=torch` instead. This will raise an error in the future!
(pid=57257) 2020-09-27 21:15:00,425	WARNING deprecation.py:30 -- DeprecationWarning: `eager` has been deprecated. Use `framework=tfe` instead. This will raise an error in the future!
(pid=57257) 2020-09-27 21:15:00,425	INFO trainer.py:632 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=57257) /home/blackbox/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
(pid=57257)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


TuneError: ('Trials did not complete', [DQN_MineRLNavigateDenseVectorObf-v0_2b461_00000])